# 4장 : 신경망 학습
이번 장의 주제는 신경망 학습이다. 여기서 $\scriptsize \textbf{학습}$이란 훈련 데이터로부터 가중치 매개변수의 최적값을 자동으로 획득하는 것을 뜻한다. 이번 장에서는 신경망이 학습할 수 있도록 해주는 지표인 $\scriptsize \textbf{손실 함수}$ 를 소개한다. 이 손실 함수의 결괏값을 가장 작게 만드는 가중치 매개변수를 찾는 것이 학습의 목표이다. 이번 장에서는 손실 함수의 값을 가급적 작게 만드는 기법으로, 함수의 기울기를 활용하는 경사법을 소개한다.

## 4.1 데이터에서 학습한다!
신경망의 특징은 데이터를 보고 학습할 수 있다는 점이다. 데이터에서 학습한다는 것은 가중치 매개변수의 값을 데이터를 보고 자동으로 결정한다는 뜻이다. 아주 멋진 소식이다! 만약 모든 매개변수를 수작업으로 결정해야 한다고 상상해보자. 생각만 해도 끔찍하다. 이를테면 2장의 퍼셉트론 예에서 진리표를 보면서 사람이 수작업으로 매개변수 값을 설정하였었다. 하지만 이때는 매개변수가 겨우 3개였다. 자, 그렇다면 실제 신경망에서는 매개변수가 몇 개나 될것인가? 정답은 수천에서 수만이다. 나아가 층을 깊게 한 딥러닝 정도 되면 그 수는 수억에 이를 수도 있다.(($\scriptsize \text{거대 언어 모델}^{\text{Large Language Model}}$)) 이쯤 되면 (아니 훨씬 전부터) 매개변수를 수작업으로 정한다는 것은 아예 불가능하다. 이번 장에서는 신경망 학습 (데이터로부터 매개변수의 값을 정하는 방법)에 대해서 설명하고 파이썬으로 MNIST 데이터셋의 손글씨 숫자를 학습하는 코드를 구현해보자.

### 4.1.1 데이터 주도 학습
기계학습은 데이터가 생명이다. 데이터에서 답을 찾고 데이터에서 패턴을 발견하고 데이터로 이야기를 만드는, 그것이 바로 기계학습이다. 데이터가 없으면 아무것도 시작되지 않는다. 그래서 기계학습의 중심에는 $\scriptsize \textbf{데이터}$가 존대한다. 이처럼 데이터가 이끄는 접근 방식 덕에 사람 중심 접근에서 벗어날 수 있었다.     

그런데 보통 어떤 문제를 해결하려 들 때, 특히 어떤 패턴을 찾아내야 할 때는 사람이 이것저것 생각하고 답을 찾는 것이 일반적이다. "이 문제는 아무래도 이런 규칙성이 있는 것 같아", "아니, 근본 원인은 다른 데 있을지도 몰라"와 같이 사람의 경험과 직관을 단서로 시행착오를 거듭하며 일을 진행한다. 반면 기계학습에서는 사람의 개입을 최소화하고 수집한 데이터로부터 패턴을 찾으려 시도한다. 게다가 신경망과 딥러닝은 기존 기계학습에서 사용하던 방법보다 사람의 개입을 더욱 배제할 수 있게 해주는 중요한 특성을 지녔다.     

구체적인 문제를 하나 생각해보자. 가령 이미지에서 '5'라는 숫자를 인식하는 프로그램을 구현한다고 해보자. [그림4-1]과 같은 자유분방한 손글씨 이미지를 보고 5인지 아닌지를 알아보는 프로그램을 구현하는 것이 목표이다. 자, 비교적 단순해 보이는 문제이다. 그런데 당장 머릿속에 떠오르는 알고리즘이 있는가?

'5'를 제대로 분류하는 프로그램을 직접 고안해 설계하기란 의외로 어려운 문제임을 알 수 있다. 사람이라면 어렵지 않게 인식하지만, 그 안에 숨은 규칙성을 명확한 로직으로 풀기는 만만치 않다. 사실 [그림4-1]을 잘 보면, 사람마다 버릇이 달라 '5'를 특징짓는 규칙을 찾기도 쉽지 않고 시간도 오래 걸릴 것 같다는 느낌이 들 것이다.      

이쯤 되면 '5'를 인식하는 알고리즘을 밑바닥부터 '설계하는' 대신, 주어진 데이터를 잘 활용해서 해결하고 싶어질 것이다. 그런 방법의 하나로, 이미지에서 $\scriptsize\textbf{특징}^{\text{feature}}$ 을 추출하고 그 특징의 패턴을 기계학습 기술로 학습하는 방법이 있다. 여기서 말하는 특징은 입력 데이터(입력 이미지)에서 본질적인 데이터(중요한 데이터)를 정확하게 추출할 수 있도록 설계된 변환기를 가리킨다. 이미지의 특징은 보통 벡터로 기술하고, 컴퓨터 비전 분야에서는 SIFT, SURF, HOG 등의 특징을 많이 사용한다. 이런 특징을 사용하여 이미지 데이터를 벡터로 변환하고, 변환된 벡터를 가지고 지도 학습 방식의 대표 분류 기법인 SVM, KNN 등으로 학습할 수 있다.       


이와 같은 기계학습에서는 모아진 데이터로부터 규칙을 찾아내는 역할을 '기계'가 담당한다. $\scriptsize\text{무}^{\text{zero}}$로부터 알고리즘을 설계하는 것보다 효율이 높아 문제를 해결해야 하는 사람의 부담도 덜어준다. 다만, 이미지를 벡터로 변환할 때 사용하는 특징은 여전히 '사람'이 설계하는 것임에 주의해야 한다. 이 말은 문제에 적합한 특징을 쓰지 않으면 (혹은 특징을 설계하지 않으면) 좀처럼 좋은 결과를 얻을 수 없다는 뜻이다. 예를 들어 개의 얼굴을 구분하려 할 때는 숫자를 인식할 때와는 다른 특징을 '사람'이 생각해야 할지도 모른다. 즉, 특징과 기계학습을 활용한 접근에도 문제에 따라서는 '사람'이 적절한 특징을 생각해내야 하는 것이다.